## Tb TF KO expression data wrangling

Created by Emanuel Flores and Adrian Jinich. 

In [ ]:
import pandas as pd 

# black magic for code style 
%load_ext blackcellmagic

Welcome! In this notebook we're going to do some simple data selection in order to make a dataset that we can work with later on. The data we'll be working with is fold-change (FC) data using single transcription factor (TF) overexpression. What this means is that they measured gene expression of all the transcriptome in normal conditions and contrasted that with the conditions of overexpressing each TF in the network, and they computed the $\mathrm{log} \left(\frac{g_{i, KO}}{g_i} \right)$, where $g_i$ is the relative level of expression of gene $i$. We're going to merge this dataset with annotation from the oxidorreductases data. 

First, we're going to load the fold-change data. 

In [ ]:
# Using pandas to load data.
fc_xl = pd.read_excel('~/Downloads/fold_changes_SI_Table_2.xlsx')

In [ ]:
# take a look
fc_xl.head(3)

In [ ]:
fc_xl.shape

We can see that there are some weird entries with NaN values. Let's filter only the FC dat.a 

## Filter only fold-change values 

Because the dataset contains p-values and ChIP-seq binding information,we'll filter the dataset to get only FC data. 

In [ ]:
# Getting column names for fold-change data. 
fc_columns = [col for col in fc_xl.columns.to_list() if 'FC' in col]

We know that there are 206 TFs, let's make sure we got all of the data. 

In [ ]:
# Make sure we got 206 columns 
len(fc_columns)

In [ ]:
# Filtering annotation data 
annot = fc_xl.iloc[:, :4]

annot.head()

In [ ]:
# Merging column names with annotation or FC data into a list
cols = annot.columns.to_list() + fc_columns

In [ ]:
# Selection of fold-change data using fancy indexing on columns 
fc_data = fc_xl[cols]

In [ ]:
# take a look 
fc_data.head(3)

In [ ]:
fc_data.shape

Nice, now we can save the dataset. 

In [ ]:
#fc_data.to_csv('../data/fold_change_tf_ko.csv', index = False)

### Load redox annotation data

Now, we want to integrate the fold-change dataset with the redox annotation, 

In [ ]:
redox = pd.read_excel('~/Downloads/redox_uniprot_myco.xlsx')

In [ ]:
redox['redox_enzyme'] = 1

Let's make an outer join keeping all the data from both dataframes. 

In order to do that, we'll first have to change the column name of the FC data to Rv_ID, to join on that column as an anchor. 

In [ ]:
# Change to have the locus name, Rv_ID as an anchor 
#fc_data.rename(columns = {'Name': 'gene_name', 'ID': 'Rv_ID'}, inplace = True)

# Change redox table column names for consistency 
redox.rename(
    columns={"Gene names": "alternative_gene_names", "Name": "redox_gene_name", 
             'Function': 'function_redox_'},
    inplace=True,
)

# Delete organism column because we know we're working with TB 
del(redox['Organism'])

In [ ]:
# Confirm the only anchor column is the locus tag 
common_cols = list(set(redox.columns.tolist()).intersection(fc_data.columns.tolist()))

common_cols

All right, we're ready to do the merge. We'll do a left outer join because we know (after some analysis) that there are some genes in the redox dataset not in the fold change dataset. 

In [ ]:
# Get genes not in FC dataset
set(redox.Rv_ID.values) - set(fc_data.Rv_ID.values).intersection(set(redox.Rv_ID.values))

In [ ]:
df = pd.merge(fc_data, redox, on = 'Rv_ID', how = 'left')

In [ ]:
df.shape

Finally, we only need to populate some annotation entries with zeros. This will help later for classification and visualization purposes. 

In [ ]:
# Fill the entries of the non-redox annotated proteins with 0s
df.redox_enzyme.fillna(0, inplace = True)
df.UK_score_4.fillna(0, inplace = True)
df.Annotation.fillna('None', inplace = True)
df.Annotation_int.fillna(0, inplace = True)

In [ ]:
# Save dataset 
df.to_csv('../data/fold_change_tf_ko_plus_redox_annot.csv', index = False)

In [ ]:
%load_ext watermark

%watermark -v -p pandas